In [1]:
# Selenium 사용
! pip install selenium

# BeautifulSoup 가져오기
! pip install bs4
from bs4 import BeautifulSoup

# 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # 스크롤
import csv
import time
import codecs
import nltk, re

In [2]:
# find_element_by_name을 통해 아이디/비밀번호 input 태그를 잡아주고, 값을 입력하기

driver = webdriver.Chrome('/Users/82107/Desktop/Selenium/chromedriver')
driver.get('https://klue.kr')

# Klue 로그인 버튼 누르기
driver.find_element_by_xpath('/html/body/app-root/menubar/div/ul/menubar-guest/span[2]').click()

## 아이디/비밀번호를 입력해준다.
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[1]').send_keys('alsgudjoo')
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[2]').send_keys('alsgud12')

# 로그인 버튼 누르기
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/button').click()

# x_path를 copy한 담에 paste하기만 해주면 된다. 

In [3]:
# 엑셀 파일 만들기
with open('클루 강의 평가(1).csv','w', encoding= 'utf-8', newline= '') as employee_file :
    employee_writer = csv.writer(employee_file)
    employee_writer.writerow(['Number','Class','Classification','Time','Credit','Semester', 'Code', 'Professor', 'Comment', 'Stars', 'Overall'])
 
    klue_site = []

    for a in range(2) :
        raw = 'https://klue.kr/lecture/{}'.format(a)
        klue_site.append(raw)

    b = klue_site
    
    for a in b :
        driver.get(a)
        
        try :
            # Alert 창 대응하기
            driver.switch_to.alert.accept()
            driver.switch_to.alert.accept()
        
        except :
        
            # 스크롤하기
            SCROLL_PAUSE_TIME = 0.5

            # Get scroll height
            last_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                # Scroll down to bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(SCROLL_PAUSE_TIME)

                # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
    
            html = driver.page_source
            klue = BeautifulSoup(html, 'html.parser')
            klue_td = klue.findAll('td')
    
            Number = a.replace("https://klue.kr/lecture/","")
            Class = klue.find(class_= 'lecture-index-main-title').get_text()
            Classification = klue_td[2].get_text()
            Time = klue_td[3].get_text()
            Credit = klue_td[4].get_text()
            Semester = klue.find('td').get_text()
            Code = klue_td[1].get_text()
            Professor = klue.find(class_= 'lecture-title-professor').get_text()
            Overall = klue.find(class_= 'lecture-main-rate-point').get_text()
            klue_Stars_Comment = klue.findAll(class_='lecture-eval-content')
            for i in klue_Stars_Comment :
                klue_Comment = i.find(class_='lecture-eval-content-context').get_text()
                Comment = klue_Comment.replace('\n','')
                klue_stars = re.findall("width\:\s(\d+)px", str(i))[0]
                Stars = int(klue_stars)/26
                employee_writer.writerow([Number, Class, Classification, Time, Credit, Semester, Code, Professor, Comment, Stars, Overall])